## Importing required libraries


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pandas as pd
import numpy as np
import quandl, math
import datetime



## To plot within notebook

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10

## for normalizing data (Feature normalization)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))





## Collect the data and read the file

In [ ]:
df = pd.read_csv('NSE-TATAGLOBAL.csv')
df.head()

In [ ]:


# Aqquirung the data
#quandl.ApiConfig.api_key = '2sn6YtKZsAaoAdfyHzfX'
#data = quandl.get('WIKI/FB')
# Removing unnecessary data
#data = data[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']];
#print(data.head(10).round(1))
#print('\n')


## Creating dataframe with date and the target variable

In [ ]:


data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])

# copying the Close value along with time into new dataframe
for i in range(0,len(data)):
     new_data['Date'][i] = data['Date'][i]
     new_data['Close'][i] = data['Close'][i]

new_data.head()



In [ ]:
# setting date as an index and droping the Date column
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

new_data.head()



## Creating train and test sets

In [ ]:

dataset = new_data.values

# spliting the dataset 0-987 in training set, rest in validation set
train = dataset[0:987,:]
valid = dataset[987:,:]

print("Length of training data ->> ",len(train))
print("Length of testing data ->> ",len(valid))

In [ ]:


#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
    
x_train, y_train = np.array(x_train), np.array(y_train)


In [ ]:
# Reshaping the training data to add extra dimension for batch training purpose
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))


## Create and fit the LSTM network

In [ ]:


model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
# model training
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

In [ ]:


# predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)




## Creating training rows (60 datapoints in one training row)

In [ ]:

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)


In [ ]:

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms



## Plotting the predicated values along with original values

In [ ]:

train = new_data[:987]
valid = new_data[987:]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])